In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import keras_tuner
import keras

In [2]:
csv_file_path = './airbnb_dataset/milestone3.csv'
df = pd.read_csv(csv_file_path)
pd.set_option('display.max_columns', None)
df.head()

,log_price,accommodates,bathrooms,host_response_rate,bedrooms,beds,cleaning_fee_True,host_has_profile_pic_t,host_identity_verified_t,instant_bookable_t,property_type_encoded,room_type_encoded,bed_type_encoded,cancellation_policy_encoded,city_encoded,des_sentiment_analysis,name_sentiment_analysis
0,5.010635,-0.072621,-0.404046,0.296013,-0.312048,-0.566461,1,1,1,0,-0.245313,0.876836,0.168067,1.021281,-0.434017,0.216242,-0.435248
1,5.129899,1.783653,-0.404046,0.296013,2.034955,1.027816,1,1,0,1,-0.245338,0.876830,0.168044,1.021245,-0.434042,0.837002,-1.788095
2,4.976734,0.855516,-0.404046,0.296013,-0.312048,1.027816,1,1,1,1,-0.245305,0.876838,0.168074,-0.593873,-0.434009,0.808508,-0.133695
3,6.620073,0.391448,-0.404046,0.296013,0.861454,0.230678,1,1,1,0,0.157557,0.876747,0.167757,-1.013554,2.675098,0.213627,-0.926212
4,4.744932,-0.536689,-0.404046,0.296013,-1.485549,-0.566461,1,1,1,1,-0.245255,0.876851,0.168119,-0.593754,1.412368,-0.101872,-0.095776


In [3]:
features = [col for col in df.columns if col != 'log_price' and df[col].dtype in ['int64', 'float64']]
X = df[features]
y = df['log_price']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Train with NN

In [5]:
# Uncomment below if tensorflow is not installed already
#%pip install tensorflow
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [6]:
# Build Neural Network
nn_model = Sequential([
    Dense(32, activation='relu',input_dim=X_train.shape[1]),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1)
])

nn_model.compile(optimizer='adam', loss='mean_squared_error')

In [7]:
# Train the model
history = nn_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
1849/1849 [==============================] - 1s 517us/step - loss: 0.6866 - val_loss: 0.2300
Epoch 2/20
1849/1849 [==============================] - 1s 450us/step - loss: 0.2197 - val_loss: 0.2146
Epoch 3/20
1849/1849 [==============================] - 1s 448us/step - loss: 0.2087 - val_loss: 0.2088
Epoch 4/20
1849/1849 [==============================] - 1s 432us/step - loss: 0.2036 - val_loss: 0.2039
Epoch 5/20
1849/1849 [==============================] - 1s 451us/step - loss: 0.2001 - val_loss: 0.2107
Epoch 6/20
1849/1849 [==============================] - 1s 432us/step - loss: 0.1973 - val_loss: 0.1981
Epoch 7/20
1849/1849 [==============================] - 1s 443us/step - loss: 0.1954 - val_loss: 0.1964
Epoch 8/20
1849/1849 [==============================] - 1s 433us/step - loss: 0.1942 - val_loss: 0.1966
Epoch 9/20
1849/1849 [==============================] - 1s 430us/step - loss: 0.1926 - val_loss: 0.1960
Epoch 10/20
1849/1849 [==============================] - 1s 430u

In [8]:
# Evaluate the model
mse = nn_model.evaluate(X_test, y_test)
print(f"Mean Squared Error on Test Set: {mse}")

463/463 [==============================] - 0s 265us/step - loss: 0.1920
Mean Squared Error on Test Set: 0.19204019010066986


### Hyper Tuning

In [9]:
def build_hp_model(hp):
    model = Sequential()
    for i in range(hp.Int('num_layers', 2, 5)):
        model.add(Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=128, step=32),
                        activation='relu'))
    model.add(Dense(1))
    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='mean_squared_error',
                  metrics=['mean_squared_error'])
    return model

In [10]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='loss',
    min_delta=0.001,
    patience=5,
    mode='auto',
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0
)

model_checkpoint = keras.callbacks.ModelCheckpoint(
    filepath='checkpoints',
    monitor='loss',
    save_best_only=True,
    save_weights_only= False,
    mode='auto'
)

In [11]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_hp_model,
    objective='val_mean_squared_error',
    max_trials=10,
    seed=10,
    executions_per_trial=3,
    overwrite=True,
    tune_new_entries=True,
    allow_new_entries=True,
    max_consecutive_failed_trials=1
)

In [12]:
# Start searching
tuner_search = tuner.search(X_train, y_train, epochs=20, callbacks=[early_stopping, model_checkpoint], validation_split=0.2)

Trial 10 Complete [00h 00m 51s]
val_mean_squared_error: 0.1850948085387548

Best val_mean_squared_error So Far: 0.18366547922293344
Total elapsed time: 00h 08m 33s


In [13]:
# View summary of searching
# tuner.results_summary()

In [18]:
# View best hyperparameters
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best hyperparameters found were: ", best_hyperparameters.values)

Best hyperparameters found were:  {'num_layers': 5, 'units_0': 96, 'learning_rate': 0.0005674460859990781, 'units_1': 96, 'units_2': 96, 'units_3': 32, 'units_4': 32}


In [19]:
best_model = tuner.get_best_models(num_models=1)[0]
mse = best_model.evaluate(X_test, y_test)
print(f"Mean Squared Error on Test Set: {mse}")

463/463 [==============================] - 0s 328us/step - loss: 0.1897 - mean_squared_error: 0.1897
Mean Squared Error on Test Set: [0.18970094621181488, 0.18970094621181488]


In [16]:
# View best model
# best_model = tuner.get_best_models()[0]
# best_model.summary()


In [17]:
# Evaluate the best model
# loss, acc = best_model.evaluate(X_test, y_test)
# print('ACC: ', acc)
# print('LOSS: ', loss)